In [103]:
from labeler import *

data = getLabels("JawClench_labels_Ansh_12-02-21-1918.txt")
startTime = data[0]
labels = data[1]
data = getData("OpenBCI-RAW-2021-12-02_19-19-53.txt",labels,startTime)

data2 = getLabels("JawClench_labels_Ansh_12-02-21-1930.txt")
startTime = data2[0]
labels = data2[1]
data2 = getData("OpenBCI-RAW-2021-12-02_19-31-12.txt", labels, startTime)

data3 = getLabels("Recordings\Spring_2022\Evan_JawClench_1.txt")
startTime = data3[0]
labels = data3[1]
data3 = getData("Recordings\Spring_2022\OpenBCISession_2022-02-16_Evan_JawClench_2\OpenBCI-RAW-2022-02-16_19-30-28.txt", labels, startTime)

data = np.append(data, data2, axis=0).tolist()
data = np.append(data, data3, axis=0).tolist()


for i in range (len(data)):
    for j in range(len(data[i])):
        if j < len(data[i])-1:
            data[i][j] = (float)(data[i][j])
        else:
            data[i][j] = (int)(data[i][j])
data = np.array(data)

2021-12-02 19:19:55.073396
2021-12-02 19:31:14.177762
2022-02-16 19:30:30.103622


In [104]:
from sklearn import svm
bsize = 200
data = np.array(data)
X = np.zeros((len(data)//bsize, 4))
y = np.zeros(len(data)//bsize)
for i in range(len(data)//bsize):
    curX = data[bsize*i: bsize*(i+1), :-1]
    curY = data[bsize*i: bsize*(i+1), -1:]
    y[i] = np.rint(np.mean(curY))
    averages = (np.mean(curX, axis=0, keepdims=True))
    mean_deviation = np.mean(curX - np.matmul(np.ones((bsize, 1)), (averages)), axis=0)
    X[i] = mean_deviation

In [105]:
print(X.shape)
print(y.shape)

(205, 4)
(205,)


In [106]:
from sklearn.model_selection import train_test_split

X_res, X_test, y_res, y_test = train_test_split(X, y, test_size=0.3, random_state = 4)
X = X_res
y = y_res
model = svm.SVC(probability=True)
model.fit(X, y)

SVC(probability=True)

In [107]:
correct = 0
total = 0

# X = np.concatenate((X_test, X))
# y = np.concatenate((y_test, y))

val_error = np.mean(np.abs(model.predict(X_test) - y_test))
print(val_error)

X = np.concatenate((X_test, X_res))
y = np.concatenate((y_test, y_res))

overall_error = np.mean(np.abs(model.predict(X) - y))
print(overall_error)
        

0.3387096774193548
0.3804878048780488
